In [54]:
import struct
from datetime import datetime
from collections import defaultdict

import pandas as pd

import sys  
sys.path.insert(0, '/home/axlan/src/sainlogic-sdr/gr-sainlogic/python')
from sainlogic_parser import get_measurements

filename = '../out/sainlog_log_20210520_213311.bin'

with open(filename, 'rb') as fd:
    packed_data = fd.read()

data = []

iter = struct.iter_unpack('I16B', packed_data)

for unpacked in iter:
    msg_data = unpacked[1:]
    values = get_measurements(msg_data)
    if values is None:
        values = {}
    values['time'] = datetime.utcfromtimestamp(unpacked[0])
    data.append(values)

df = pd.DataFrame(data)
df.set_index('time', inplace=True)

print(df)

print(f'{sum(df["temp"].isna())} CRC Failures')



                     temp  humidity  wind_dir  avr_wind  gust_wind
time                                                              
2021-05-20 21:33:23  65.2      35.0      78.0       2.5        5.6
2021-05-20 21:33:39  64.7      35.0     150.0       3.8        6.6
2021-05-20 21:33:55  64.1      36.0     139.0       2.4        3.0
2021-05-20 21:34:11  64.3      37.0     152.0       1.9        2.6
2021-05-20 21:34:27  65.2      36.0      63.0       2.1        2.6
...                   ...       ...       ...       ...        ...
2021-05-20 23:31:14  63.7      43.0     153.0       3.4        4.3
2021-05-20 23:31:30  63.9      43.0     128.0       2.5        4.0
2021-05-20 23:31:46  63.8      43.0      97.0       3.3        4.6
2021-05-20 23:32:02  62.6      44.0      59.0       4.6        6.3
2021-05-20 23:32:18  63.2      44.0      48.0       2.7        4.0

[425 rows x 5 columns]
3 CRC Failures


In [55]:
import plotly.express as px
import numpy as np

time_delta_secs = np.diff(df.index.values).astype('timedelta64[s]').astype(np.int32)

fig = px.histogram(time_delta_secs, histnorm='percent', nbins=20)
fig.show()





In [56]:
fig = px.line(df, y='temp')
fig.show()


In [57]:
fig = px.line(df, y='avr_wind')
fig.show()

In [58]:
fig = px.line(df, y='humidity')
fig.show()